In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our input dataset
loans_df = pd.read_csv('loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [2]:
#we must make sure that none of our categorical variables require bucketing


# Generate our categorical variable list
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [3]:
# Check the unique value counts to see if binning is required
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [4]:
#we have reason to leave the “Years_in_current_job”
#column alone because we don’t want to bucket common
#values together and cause confusion in the model.

#categorical variables are ready for encoding


In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(loans_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
# Remove loan status target from features data
y = loans_df.Loan_Status_Fully_Paid
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")


 Random forest predictive accuracy: 0.849


In [9]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
854/854 [==============================] - 1s 734us/step - loss: 0.4176 - accuracy: 0.8415
Epoch 2/50
854/854 [==============================] - 1s 691us/step - loss: 0.3856 - accuracy: 0.8488
Epoch 3/50
854/854 [==============================] - 1s 721us/step - loss: 0.3818 - accuracy: 0.8493
Epoch 4/50
854/854 [==============================] - 1s 714us/step - loss: 0.3794 - accuracy: 0.8492
Epoch 5/50
854/854 [==============================] - 1s 737us/step - loss: 0.3779 - accuracy: 0.8493
Epoch 6/50
854/854 [==============================] - 1s 720us/step - loss: 0.3769 - accuracy: 0.8494
Epoch 7/50
854/854 [==============================] - 1s 700us/step - loss: 0.3760 - accuracy: 0.8492
Epoch 8/50
854/854 [==============================] - 1s 697us/step - loss: 0.3749 - accuracy: 0.8498
Epoch 9/50
854/854 [==============================] - 1s 692us/step - loss: 0.3740 - accuracy: 0.8498
Epoch 10/50
854/854 [==============================] - 1s 688us/step - loss: 0.373

In [10]:
#19.6.1 Checkpoints Are Not Just for Video Games

#1. Import dependencies.
#2. Import the input dataset.
#3. Generate categorical variable list.
#4. Create a OneHotEncoder instance.
#5. Fit and transform the OneHotEncoder.
#6. Add the encoded variable names to the DataFrame.
#7. Merge one-hot encoded features and drop the originals.
#8. Split the preprocessed data into features and target arrays.
#9. Split the preprocessed data into training and testing dataset.
#10. Create a StandardScaler instance.
#11. Fit the StandardScaler.
#12. Scale the data.
#13. Define the model.
#14. Add first and second hidden layers.
#15. Add the output layer.
#16. Check the structure of the model.

In [11]:
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                888       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
854/854 [==============================] - 1s 688us/step - loss: 0.3602 - accuracy: 0.8524
Epoch 2/100
136/854 [===>..........................] - ETA: 0s - loss: 0.3517 - accuracy: 0.8559
Epoch 00002: saving model to checkpoints/weights.02.hdf5
854/854 [==============================] - 1s 722us/step - loss: 0.3600 - accuracy: 0.8522
Epoch 3/100
286/854 [=========>....................] - ETA: 0s - loss: 0.3563 - accuracy: 0.8536
Epoch 00003: saving model to checkpoints/weights.03.hdf5
854/854 [==============================] - 1s 738us/step - loss: 0.3596 - accuracy: 0.8529
Epoch 4/100
429/854 [==============>...............] - ETA: 0s - loss: 0.3589 - accuracy: 0.8521
Epoch 00004: saving model to checkpoints/weights.04.hdf5
854/854 [==============================] - 1s 725us/step - loss: 0.3593 - accuracy: 0.8526
Epoch 5/100
568/854 [==================>...........] - ETA: 0s - loss: 0.3603 - accuracy: 0.8519
Epoch 00005: saving model to checkpoints/weights.05.hdf5
854/854 

854/854 [==============================] - 1s 708us/step - loss: 0.3551 - accuracy: 0.8543
Epoch 38/100
374/854 [============>.................] - ETA: 0s - loss: 0.3553 - accuracy: 0.8522
Epoch 00038: saving model to checkpoints/weights.38.hdf5
854/854 [==============================] - 1s 698us/step - loss: 0.3548 - accuracy: 0.8533
Epoch 39/100
496/854 [================>.............] - ETA: 0s - loss: 0.3560 - accuracy: 0.8513
Epoch 00039: saving model to checkpoints/weights.39.hdf5
854/854 [==============================] - 1s 715us/step - loss: 0.3550 - accuracy: 0.8524
Epoch 40/100
666/854 [======================>.......] - ETA: 0s - loss: 0.3532 - accuracy: 0.8544
Epoch 00040: saving model to checkpoints/weights.40.hdf5
854/854 [==============================] - 1s 703us/step - loss: 0.3547 - accuracy: 0.8538
Epoch 41/100
822/854 [===========================>..] - ETA: 0s - loss: 0.3549 - accuracy: 0.8534
Epoch 00041: saving model to checkpoints/weights.41.hdf5
854/854 [=======

854/854 [==============================] - 1s 745us/step - loss: 0.3527 - accuracy: 0.8539
Epoch 74/100
610/854 [====================>.........] - ETA: 0s - loss: 0.3498 - accuracy: 0.8556
Epoch 00074: saving model to checkpoints/weights.74.hdf5
854/854 [==============================] - 1s 738us/step - loss: 0.3523 - accuracy: 0.8545
Epoch 75/100
744/854 [=========================>....] - ETA: 0s - loss: 0.3527 - accuracy: 0.8542
Epoch 00075: saving model to checkpoints/weights.75.hdf5
854/854 [==============================] - 1s 691us/step - loss: 0.3524 - accuracy: 0.8549
Epoch 76/100
854/854 [==============================] - 1s 693us/step - loss: 0.3525 - accuracy: 0.8545
Epoch 77/100
 79/854 [=>............................] - ETA: 0s - loss: 0.3578 - accuracy: 0.8525
Epoch 00077: saving model to checkpoints/weights.77.hdf5
854/854 [==============================] - 1s 698us/step - loss: 0.3526 - accuracy: 0.8546
Epoch 78/100
219/854 [======>.......................] - ETA: 0s - l

In [14]:
#Define the model - deep neural net
number_input_features = len(X_train_scaled[0])

In [15]:
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

In [16]:
nn_new = tf.keras.models.Sequential()

In [18]:
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

ValueError: Shapes (36, 8) and (36, 24) are incompatible

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 0s - loss: 1.3394 - accuracy: 0.2073
Loss: 1.3393936157226562, Accuracy: 0.20733582973480225


In [20]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [21]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 0s - loss: 1.3394 - accuracy: 0.2073
Loss: 1.3393936157226562, Accuracy: 0.20733582973480225
